In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []

    for member in root.findall('object'):
        label = member.find('name').text
        bbox = member.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        annotations.append({
            "label": label,
            "bbox": [xmin, ymin, xmax, ymax]
        })

    return annotations

def load_images_and_annotations(images_dir, annotations_dir):
    all_images = []
    all_annotations = []

    for image_file in os.listdir(images_dir):
        if image_file.endswith('.png'):  # or other image formats if applicable
            image_path = os.path.join(images_dir, image_file)
            annotation_path = os.path.join(annotations_dir, image_file.replace('.png', '.xml'))

            # Load image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Parse annotation
            annotation = parse_xml(annotation_path)

            all_images.append(image)
            all_annotations.append(annotation)

    return all_images, all_annotations

# Example usage
images_dir = 'images'
annotations_dir = 'annotations'
images, annotations = load_images_and_annotations(images_dir, annotations_dir)


In [ ]:
import albumentations as A

def get_augmentation_pipeline():
    # Define a series of augmentations
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(p=1),
        A.Resize(width=300, height=300, p=1)  # Resize images to a uniform size
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Example usage
augmentation_pipeline = get_augmentation_pipeline()


In [ ]:
class_counts = {
    'trafficlight': 170,  # The count of 'trafficlight' instances in the original dataset
    'stop': 91,           # The count of 'stop' instances
    'speedlimit': 783,    # The count of 'speedlimit' instances
    'crosswalk': 200      # The count of 'crosswalk' instances
}


In [ ]:
def apply_selective_augmentations(images, annotations, augmentation_pipeline, class_counts, target_count):
    augmented_images = []
    augmented_annotations = []
    augmented_class_counts = {class_name: 0 for class_name in class_counts.keys()}

    for image, annotation_list in zip(images, annotations):
        for annotation in annotation_list:
            label = annotation['label']
            # Check if the class is underrepresented and needs augmentation
            if class_counts[label] + augmented_class_counts[label] < target_count:
                bbox = annotation['bbox']

                # Apply augmentation multiple times per image
                for _ in range(3):  # Adjust this number as needed
                    augmented = augmentation_pipeline(image=image, bboxes=[bbox], class_labels=[label])
                    augmented_image = augmented['image']
                    augmented_bbox = augmented['bboxes'][0] if augmented['bboxes'] else None

                    if augmented_bbox:
                        # Update annotation and add to lists
                        new_annotation = annotation.copy()
                        new_annotation['bbox'] = augmented_bbox
                        augmented_images.append(augmented_image)
                        augmented_annotations.append(new_annotation)
                        augmented_class_counts[label] += 1

    return augmented_images, augmented_annotations, augmented_class_counts

# Example usage
most_represented_class_count = max(class_counts.values())
augmented_images, augmented_annotations, augmented_counts = apply_selective_augmentations(
    images, annotations, augmentation_pipeline, class_counts, most_represented_class_count)



In [ ]:
def save_image(image, save_dir, file_name):
    save_path = os.path.join(save_dir, file_name)
    # Convert image from RGB back to BGR before saving with OpenCV
    cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

def create_xml_annotation(annotation, file_name, folder_name, image_shape):
    root = ET.Element("annotation")

    folder = ET.SubElement(root, "folder")
    folder.text = folder_name

    filename = ET.SubElement(root, "filename")
    filename.text = file_name

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(image_shape[1])
    ET.SubElement(size, "height").text = str(image_shape[0])
    ET.SubElement(size, "depth").text = str(image_shape[2])

    object_tag = ET.SubElement(root, "object")
    ET.SubElement(object_tag, "name").text = annotation['label']
    ET.SubElement(object_tag, "pose").text = "Unspecified"
    ET.SubElement(object_tag, "truncated").text = "0"
    ET.SubElement(object_tag, "difficult").text = "0"

    bbox = ET.SubElement(object_tag, "bndbox")
    ET.SubElement(bbox, "xmin").text = str(int(annotation['bbox'][0]))
    ET.SubElement(bbox, "ymin").text = str(int(annotation['bbox'][1]))
    ET.SubElement(bbox, "xmax").text = str(int(annotation['bbox'][2]))
    ET.SubElement(bbox, "ymax").text = str(int(annotation['bbox'][3]))

    tree = ET.ElementTree(root)
    return tree

def save_annotation(annotation, save_dir, file_name):
    xml_tree = create_xml_annotation(annotation, file_name, 'augmented_images', annotation['image_shape'])
    xml_tree.write(os.path.join(save_dir, file_name.replace('.png', '.xml')))

# Directories for saving augmented images and annotations
augmented_images_dir = 'augmented_images'
augmented_annotations_dir = 'augmented_annotations'

for i, (image, annotation) in enumerate(zip(augmented_images, augmented_annotations)):
    file_name = f"road{i+877}.png"
    annotation['image_shape'] = image.shape

    save_image(image, augmented_images_dir, file_name)
    save_annotation(annotation, augmented_annotations_dir, file_name)


In [ ]:
from collections import Counter

# Assuming augmented_annotations is a list of annotation dictionaries
augmented_class_counts = Counter()

for annotation in augmented_annotations:
    augmented_class_counts[annotation['label']] += 1

print(augmented_class_counts)


In [ ]:
import os
import xml.etree.ElementTree as ET
from collections import Counter

annotations_path = 'annotations'
annotations_files = os.listdir(annotations_path)

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []

    for member in root.findall('object'):
        label = member.find('name').text
        bbox = member.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        annotations.append({
            "label": label,
            "bbox": [xmin, ymin, xmax, ymax]
        })

    return annotations

all_annotations = [parse_xml(os.path.join(annotations_path, file)) for file in annotations_files]


class_counts = Counter()
for annotation_list in all_annotations:
    for annotation in annotation_list:
        class_counts[annotation['label']] += 1

class_counts
